In [3]:
from __future__ import print_function, division

import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob

from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import time

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []
    i = 0
    for file in glob.glob("pokemon_data/*.mid"):
        midi = converter.parse(file)
        i = i + 1
        if i > 5: break
        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 250

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output
  
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):

        print('[train]: Starting...')
        time_start = time.time()
        # Load and convert the data
        notes = get_notes()
        n_vocab = len(set(notes))
        X_train, y_train = prepare_sequences(notes, n_vocab)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Insert saving; The generator image is saved every 500 steps
        save_interval = 500

        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)



            if (epoch + 1) % save_interval == 0:

                # Save the generator model in a periodic basis
                model_name = './pokemon_models/pokemon_' + str(epoch)
                self.generator.save(model_name + ".h5")

                # Generate a midi file in a periodic basis
                self.generate(notes, epoch)


            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [time: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss, time.time() - time_start))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        self.generate(notes, epoch)
        self.plot_loss()


        time_end = time.time()
        print('[train]: Ended. Time elapsed: {}'.format(time_end - time_start))
        
    def generate(self, input_notes, epoch):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(set(item for item in notes))
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        pred_notes = [x*242+242 for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        

        file_name = './pokemon_output/pokemon_' + str(epoch)
        create_midi(pred_notes, file_name)
        time_end = time.time()
        print('[generate midi]: Ended. Time elapsed: {}'.format(time_end - time_start))

        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('./pokemon_output/GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=250)    
  gan.train(epochs=500, batch_size=32, sample_interval=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_5 (CuDNNLSTM)     (None, 250, 512)          1054720   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dense_15 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total para

65 [D loss: 0.588406, acc.: 67.19%] [G loss: 1.129282] [time: 21.696610]
66 [D loss: 0.591102, acc.: 64.06%] [G loss: 1.154762] [time: 21.917524]
67 [D loss: 0.611527, acc.: 67.19%] [G loss: 1.157943] [time: 22.139057]
68 [D loss: 0.576031, acc.: 73.44%] [G loss: 1.215367] [time: 22.358922]
69 [D loss: 0.753453, acc.: 54.69%] [G loss: 0.953474] [time: 22.579995]
70 [D loss: 0.551716, acc.: 70.31%] [G loss: 0.995333] [time: 22.799789]
71 [D loss: 0.532776, acc.: 71.88%] [G loss: 1.041493] [time: 23.019585]
72 [D loss: 0.604348, acc.: 68.75%] [G loss: 1.059740] [time: 23.238485]
73 [D loss: 0.514217, acc.: 73.44%] [G loss: 1.079107] [time: 23.458284]
74 [D loss: 0.573355, acc.: 67.19%] [G loss: 1.110654] [time: 23.679294]
75 [D loss: 0.635243, acc.: 65.62%] [G loss: 1.090424] [time: 23.899553]
76 [D loss: 0.467001, acc.: 76.56%] [G loss: 1.182709] [time: 24.119615]
77 [D loss: 0.572107, acc.: 68.75%] [G loss: 1.163296] [time: 24.343065]
78 [D loss: 0.505901, acc.: 75.00%] [G loss: 1.2077

177 [D loss: 0.647377, acc.: 59.38%] [G loss: 0.930281] [time: 46.394727]
178 [D loss: 0.596699, acc.: 70.31%] [G loss: 0.916744] [time: 46.616474]
179 [D loss: 0.675691, acc.: 57.81%] [G loss: 1.012301] [time: 46.838968]
180 [D loss: 0.581510, acc.: 68.75%] [G loss: 1.008848] [time: 47.058334]
181 [D loss: 0.480417, acc.: 84.38%] [G loss: 1.213946] [time: 47.282761]
182 [D loss: 0.655147, acc.: 67.19%] [G loss: 0.893947] [time: 47.502207]
183 [D loss: 0.642858, acc.: 71.88%] [G loss: 0.997156] [time: 47.723006]
184 [D loss: 0.653281, acc.: 59.38%] [G loss: 0.940645] [time: 47.940915]
185 [D loss: 0.674656, acc.: 57.81%] [G loss: 0.938088] [time: 48.161052]
186 [D loss: 0.588765, acc.: 73.44%] [G loss: 0.906611] [time: 48.381045]
187 [D loss: 0.624234, acc.: 62.50%] [G loss: 0.931746] [time: 48.599789]
188 [D loss: 0.671341, acc.: 57.81%] [G loss: 1.028337] [time: 48.824251]
189 [D loss: 0.583902, acc.: 73.44%] [G loss: 1.016665] [time: 49.045132]
190 [D loss: 0.661987, acc.: 56.25%] [

288 [D loss: 0.713296, acc.: 57.81%] [G loss: 0.800153] [time: 70.937344]
289 [D loss: 0.674052, acc.: 64.06%] [G loss: 0.970022] [time: 71.160956]
290 [D loss: 0.618635, acc.: 64.06%] [G loss: 1.874920] [time: 71.380380]
291 [D loss: 0.786964, acc.: 62.50%] [G loss: 1.279358] [time: 71.601375]
292 [D loss: 0.980229, acc.: 34.38%] [G loss: 0.851823] [time: 71.827323]
293 [D loss: 0.665257, acc.: 56.25%] [G loss: 1.012057] [time: 72.051673]
294 [D loss: 0.639245, acc.: 59.38%] [G loss: 1.337519] [time: 72.276234]
295 [D loss: 0.712777, acc.: 60.94%] [G loss: 1.024288] [time: 72.496944]
296 [D loss: 0.631691, acc.: 65.62%] [G loss: 1.071108] [time: 72.721997]
297 [D loss: 0.677220, acc.: 62.50%] [G loss: 0.968323] [time: 72.944337]
298 [D loss: 0.779609, acc.: 56.25%] [G loss: 0.851553] [time: 73.165634]
299 [D loss: 0.680684, acc.: 60.94%] [G loss: 0.840111] [time: 73.386739]
300 [D loss: 0.643054, acc.: 65.62%] [G loss: 0.963316] [time: 73.608302]
301 [D loss: 0.676593, acc.: 60.94%] [

399 [D loss: 0.699683, acc.: 40.62%] [G loss: 0.685457] [time: 95.739490]
400 [D loss: 0.709090, acc.: 35.94%] [G loss: 0.701397] [time: 95.965020]
401 [D loss: 0.696132, acc.: 40.62%] [G loss: 0.713244] [time: 96.187557]
402 [D loss: 0.700898, acc.: 39.06%] [G loss: 0.718854] [time: 96.412963]
403 [D loss: 0.693426, acc.: 59.38%] [G loss: 0.717721] [time: 96.637223]
404 [D loss: 0.688287, acc.: 56.25%] [G loss: 0.715210] [time: 96.863914]
405 [D loss: 0.691197, acc.: 57.81%] [G loss: 0.723174] [time: 97.088331]
406 [D loss: 0.694799, acc.: 54.69%] [G loss: 0.720600] [time: 97.309517]
407 [D loss: 0.686852, acc.: 60.94%] [G loss: 0.704952] [time: 97.533551]
408 [D loss: 0.705849, acc.: 46.88%] [G loss: 0.714021] [time: 97.756751]
409 [D loss: 0.702242, acc.: 46.88%] [G loss: 0.732238] [time: 97.980379]
410 [D loss: 0.715968, acc.: 40.62%] [G loss: 0.690886] [time: 98.201562]
411 [D loss: 0.697564, acc.: 48.44%] [G loss: 0.695042] [time: 98.427196]
412 [D loss: 0.707399, acc.: 35.94%] [

KeyError: 409